In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)

#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Down", "Up"]

In [3]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [4]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(130, 4, 96)
(130,)
(86, 4, 96)
(86,)


In [5]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [6]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [7]:
#Normalizing data:
x_train = x_train.astype("float")
x_test = x_test.astype("float")
merge = np.vstack((x_train, x_test))
minval = np.amin(merge)
maxval = np.amax(merge)
print(f'Min: {minval}, Max:{maxval}')
x_train = (x_train-minval)/(maxval-minval)
x_test = (x_test-minval)/(maxval-minval)

Min: -194.824, Max:221.68


In [8]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1, activation='softmax'))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [10]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 1000
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

Train on 130 samples, validate on 86 samples
Epoch 1/1000
130/130 [==============================] - 0s 90us/step - loss: 0.2403 - accuracy: 0.5692 - val_loss: 0.2394 - val_accuracy: 0.6279
Epoch 2/1000
130/130 [==============================] - 0s 67us/step - loss: 0.2415 - accuracy: 0.5692 - val_loss: 0.2382 - val_accuracy: 0.6163
Epoch 3/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2408 - accuracy: 0.6077 - val_loss: 0.2493 - val_accuracy: 0.5000
Epoch 4/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2462 - accuracy: 0.4846 - val_loss: 0.2666 - val_accuracy: 0.3837
Epoch 5/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2514 - accuracy: 0.4538 - val_loss: 0.2629 - val_accuracy: 0.3837
Epoch 6/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2492 - accuracy: 0.4615 - val_loss: 0.2589 - val_accuracy: 0.3953
Epoch 7/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2471 -

Epoch 57/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2385 - accuracy: 0.6000 - val_loss: 0.2364 - val_accuracy: 0.6163
Epoch 58/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2446 - accuracy: 0.5615 - val_loss: 0.2367 - val_accuracy: 0.6163
Epoch 59/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2430 - accuracy: 0.5615 - val_loss: 0.2376 - val_accuracy: 0.6279
Epoch 60/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2411 - accuracy: 0.6462 - val_loss: 0.2551 - val_accuracy: 0.3721
Epoch 61/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2462 - accuracy: 0.4769 - val_loss: 0.2676 - val_accuracy: 0.3837
Epoch 62/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2499 - accuracy: 0.4615 - val_loss: 0.2619 - val_accuracy: 0.3953
Epoch 63/1000
130/130 [==============================] - 0s 61us/step - loss: 0.2434 - accuracy: 0.5385 - val_loss: 0.2405 -

130/130 [==============================] - 0s 45us/step - loss: 0.2349 - accuracy: 0.6154 - val_loss: 0.2449 - val_accuracy: 0.5930
Epoch 114/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2364 - accuracy: 0.6077 - val_loss: 0.2484 - val_accuracy: 0.5465
Epoch 115/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2350 - accuracy: 0.6154 - val_loss: 0.2385 - val_accuracy: 0.6163
Epoch 116/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2396 - accuracy: 0.6231 - val_loss: 0.2368 - val_accuracy: 0.6163
Epoch 117/1000
130/130 [==============================] - 0s 44us/step - loss: 0.2417 - accuracy: 0.5769 - val_loss: 0.2368 - val_accuracy: 0.6163
Epoch 118/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2362 - accuracy: 0.6000 - val_loss: 0.2403 - val_accuracy: 0.6395
Epoch 119/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2355 - accuracy: 0.6231 - val_loss: 0.2463 - val_acc

Epoch 169/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2331 - accuracy: 0.6154 - val_loss: 0.2426 - val_accuracy: 0.5814
Epoch 170/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2338 - accuracy: 0.6077 - val_loss: 0.2379 - val_accuracy: 0.6163
Epoch 171/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2350 - accuracy: 0.6077 - val_loss: 0.2387 - val_accuracy: 0.6395
Epoch 172/1000
130/130 [==============================] - 0s 75us/step - loss: 0.2334 - accuracy: 0.6462 - val_loss: 0.2410 - val_accuracy: 0.6047
Epoch 173/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2325 - accuracy: 0.6154 - val_loss: 0.2460 - val_accuracy: 0.5930
Epoch 174/1000
130/130 [==============================] - 0s 98us/step - loss: 0.2366 - accuracy: 0.6538 - val_loss: 0.2583 - val_accuracy: 0.3721
Epoch 175/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2387 - accuracy: 0.6077 - val_loss: 0

Epoch 225/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2309 - accuracy: 0.6154 - val_loss: 0.2414 - val_accuracy: 0.6047
Epoch 226/1000
130/130 [==============================] - 0s 44us/step - loss: 0.2317 - accuracy: 0.6385 - val_loss: 0.2385 - val_accuracy: 0.6395
Epoch 227/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2330 - accuracy: 0.6538 - val_loss: 0.2393 - val_accuracy: 0.6279
Epoch 228/1000
130/130 [==============================] - 0s 59us/step - loss: 0.2313 - accuracy: 0.6308 - val_loss: 0.2445 - val_accuracy: 0.5930
Epoch 229/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2331 - accuracy: 0.6385 - val_loss: 0.2631 - val_accuracy: 0.3953
Epoch 230/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2411 - accuracy: 0.4923 - val_loss: 0.2691 - val_accuracy: 0.3837
Epoch 231/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2407 - accuracy: 0.4692 - val_loss: 0

...

Epoch 896/1000
130/130 [==============================] - 0s 67us/step - loss: 0.2288 - accuracy: 0.6615 - val_loss: 0.2585 - val_accuracy: 0.4419
Epoch 897/1000
130/130 [==============================] - 0s 52us/step - loss: 0.2265 - accuracy: 0.6462 - val_loss: 0.2532 - val_accuracy: 0.5233
Epoch 898/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2242 - accuracy: 0.6077 - val_loss: 0.2507 - val_accuracy: 0.5814
Epoch 899/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2240 - accuracy: 0.6154 - val_loss: 0.2594 - val_accuracy: 0.4651
Epoch 900/1000
130/130 [==============================] - 0s 60us/step - loss: 0.2308 - accuracy: 0.6462 - val_loss: 0.2681 - val_accuracy: 0.4186
Epoch 901/1000
130/130 [==============================] - 0s 53us/step - loss: 0.2274 - accuracy: 0.7000 - val_loss: 0.2487 - val_accuracy: 0.5814
Epoch 902/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2230 - accuracy: 0.6231 - val_loss: 0

Epoch 952/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2224 - accuracy: 0.6154 - val_loss: 0.2433 - val_accuracy: 0.5930
Epoch 953/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2231 - accuracy: 0.6154 - val_loss: 0.2441 - val_accuracy: 0.6047
Epoch 954/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2240 - accuracy: 0.6154 - val_loss: 0.2478 - val_accuracy: 0.5814
Epoch 955/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2233 - accuracy: 0.6154 - val_loss: 0.2491 - val_accuracy: 0.5814
Epoch 956/1000
130/130 [==============================] - 0s 44us/step - loss: 0.2229 - accuracy: 0.6154 - val_loss: 0.2453 - val_accuracy: 0.5814
Epoch 957/1000
130/130 [==============================] - 0s 37us/step - loss: 0.2234 - accuracy: 0.6154 - val_loss: 0.2438 - val_accuracy: 0.6047
Epoch 958/1000
130/130 [==============================] - 0s 45us/step - loss: 0.2215 - accuracy: 0.6154 - val_loss: 0

requests_with_retry encountered retryable exception: ('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')")). args: ('https://api.wandb.ai/files/peter_jozsef/eeg-up-down/609edlgm/file_stream',), kwargs: {'json': {'files': {'wandb-events.jsonl': {'offset': 0, 'content': ['{"system.cpu": 15.75, "system.memory": 61.91, "system.disk": 59.6, "system.proc.memory.availableMB": 6164.67, "system.proc.memory.rssMB": 261.74, "system.proc.memory.percent": 1.62, "system.proc.cpu.threads": 46.62, "system.network.sent": 746159, "system.network.recv": 236312, "_wandb": true, "_timestamp": 1571901279, "_runtime": 13}\n']}, 'wandb-history.jsonl': {'offset': 994, 'content': ['{"epoch": 994, "val_loss": 0.2403331897979559, "val_accuracy": 0.6279069781303406, "loss": 0.22282665234345655, "accuracy": 0.6307692527770996, "_runtime": 13.370345115661621, "_timestamp": 1571901279.6008403, "_step": 994}\n', '{"epoch": 995, "val_loss": 0.24110778750375259, "val_accuracy": 0.6395348906517029, "loss": 0.2349338

wandb: Network error (ConnectTimeout), entering retry loop. See C:\Users\jocom\PythonProjects\muse\wandb\debug.log for full traceback.
requests_with_retry encountered retryable exception: HTTPSConnectionPool(host='api.wandb.ai', port=443): Max retries exceeded with url: /files/peter_jozsef/eeg-up-down/609edlgm/file_stream (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000021AA189A948>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')). args: ('https://api.wandb.ai/files/peter_jozsef/eeg-up-down/609edlgm/file_stream',), kwargs: {'json': {'files': {'wandb-events.jsonl': {'offset': 0, 'content': ['{"system.cpu": 15.75, "system.memory": 61.91, "system.disk": 59.6, "system.proc.memory.availableMB": 6164.67, "system.proc.memory.rssMB": 261.74, "system.proc.memory.

requests_with_retry encountered retryable exception: HTTPSConnectionPool(host='api.wandb.ai', port=443): Max retries exceeded with url: /files/peter_jozsef/eeg-up-down/609edlgm/file_stream (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000021AA1D23E08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')). args: ('https://api.wandb.ai/files/peter_jozsef/eeg-up-down/609edlgm/file_stream',), kwargs: {'json': {'files': {'wandb-events.jsonl': {'offset': 0, 'content': ['{"system.cpu": 15.75, "system.memory": 61.91, "system.disk": 59.6, "system.proc.memory.availableMB": 6164.67, "system.proc.memory.rssMB": 261.74, "system.proc.memory.percent": 1.62, "system.proc.cpu.threads": 46.62, "system.network.sent": 746159, "system.network.recv": 236312, "_wandb": true, "_timestamp": 1571901279, "_runtime": 13}\n']}, 'wandb-history.jsonl': {'offset': 994, 'content': ['{"epoch": 994, "val_loss": 0.2403331897979559, "val_accuracy": 0.627906978

In [ ]:
print(model.predict(x_test[10:50,:,:]))
print(is_up_y_test[10:50])